# Quality control

In [1]:
import hail as hl
hl.init(spark_conf={'spark.driver.memory': '10g'}, tmp_dir='/home/olavur/tmp')

2021-10-27 12:05:27 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2021-10-27 12:05:28 WARN  Hail:37 - This Hail JAR was compiled for Spark 2.4.5, running with Spark 2.4.1.
  Compatibility is not guaranteed.
2021-10-27 12:05:28 WARN  Utils:66 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2021-10-27 12:05:28 WARN  Utils:66 - Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Running on Apache Spark version 2.4.1
SparkUI available at http://hms-beagle-6676655f87-9xllv:4042
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /home/olavur/experiments/2020-11-13_fargen1_exome_analysis/fargen-1-exome/notebooks/main/hail-20211027-1205-0.2.61-3c86d3ba497a.log


In [2]:
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
from bokeh.models.scales import LogScale
output_notebook()

Loading BokehJS ...

In [3]:
import pandas as pd

Load variant data.

In [4]:
BASE_DIR = '/home/olavur/experiments/2020-11-13_fargen1_exome_analysis'
mt = hl.read_matrix_table(BASE_DIR + '/data/mt/variants.mt')

In [5]:
n_variants, n_samples = mt.count()
print('Number of variants: ' + str(n_variants))
print('Number of samples: ' + str(n_samples))

Number of variants: 911929
Number of samples: 473


## Variant QC

### Variant counts

In [6]:
def variant_counts(mt):
    # Count number of variants, SNPs and indels. Only first allele in alternate allele list is considered.
    variant_counts_struct = mt.aggregate_rows(hl.struct(
        n_variants = hl.agg.count(),
        snps_fraction = hl.agg.count_where(hl.is_snp(mt.alleles[0], mt.alleles[1])) / hl.agg.count(),
        insertions_fraction = hl.agg.count_where(hl.is_insertion(mt.alleles[0], mt.alleles[1])) / hl.agg.count(),
        deletions_fraction = hl.agg.count_where(hl.is_deletion(mt.alleles[0], mt.alleles[1])) / hl.agg.count(),
        mnp_fraction = hl.agg.count_where(hl.is_mnp(mt.alleles[0], mt.alleles[1])) / hl.agg.count(),
        complex_fraction = hl.agg.count_where(hl.is_complex(mt.alleles[0], mt.alleles[1])) / hl.agg.count(),
        star_fraction = hl.agg.count_where(hl.is_star(mt.alleles[0], mt.alleles[1])) / hl.agg.count()))
    
    variant_counts_pd = pd.DataFrame(variant_counts_struct.values(), index=variant_counts_struct.keys(), columns=[''])
    return variant_counts_pd

Create a dataset where multi-allelic sites are split.

In [7]:
# NOTE: permit_shuffle is needed when there are duplicate loci, one of which has multiple alternate alleles. Apparently, this dataset has such sites.
split_mt = hl.split_multi_hts(mt, permit_shuffle=True)

In [8]:
variant_counts(split_mt)

[Stage 0:=======================================================> (72 + 2) / 74]

,
n_variants,1.106101e+06
snps_fraction,5.907046e-01
insertions_fraction,2.523793e-01
deletions_fraction,1.569161e-01
mnp_fraction,0.000000e+00
complex_fraction,0.000000e+00
star_fraction,0.000000e+00


### Variant call quality (QUAL)

Plot distribution of QUAL values. Note that a hard cut-off filter has been used on QUAL in the joint genotyping pipeline.

In [9]:
p = hl.plot.histogram(mt.qual, legend='Variant call quality (QUAL)')
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 1:=======================================================> (36 + 1) / 37]

In [10]:
p = hl.plot.histogram(hl.log10(mt.qual), legend='Variant call quality (log10 of QUAL)')
p.plot_width = 800
p.plot_height = 500
show(p)

### VQSR filters

We will inspect the VQSR tranches.

**FIXME:** These variants with `filters = None` have `VQSLOD = NaN`, as shown in the Pandas table below. What are these variants? Why have they not been assigned a VQSLOD?

In [11]:
mt = mt.transmute_rows(filters=hl.delimit(mt.filters, ','))

In [12]:
mt.aggregate_rows(hl.agg.counter(mt.filters))

[Stage 5:==================================>                     (23 + 14) / 37]

{'': 159849,
 'VQSRTrancheINDEL99.90to100.00': 53479,
 'VQSRTrancheINDEL99.00to99.90': 46133,
 'VQSRTrancheSNP99.90to100.00': 68405,
 'VQSRTrancheSNP99.00to99.90': 139682,
 'VQSRTrancheINDEL90.00to99.00': 56409,
 'VQSRTrancheSNP90.00to99.00': 111492,
 'VQSRTrancheSNP50.00to75.00': 102544,
 'VQSRTrancheINDEL50.00to75.00': 24351,
 'VQSRTrancheSNP75.00to90.00': 93294,
 'VQSRTrancheINDEL75.00to90.00': 56291}

We will calculate some mean QC statistics for each VQSR tranch.

In [13]:
# Calculate variant statistics.
mt = hl.variant_qc(mt)

# Get rows table.
rows_ht = mt.rows()

# Aggregate.
result = (rows_ht.group_by(rows_ht.filters)
         .aggregate(mean_gq = hl.agg.filter(~hl.is_nan(rows_ht.variant_qc.gq_stats.mean), hl.agg.mean(rows_ht.variant_qc.gq_stats.mean)),
                   mean_dp = hl.agg.filter(~hl.is_nan(rows_ht.variant_qc.dp_stats.mean), hl.agg.mean(rows_ht.variant_qc.dp_stats.mean)),
                   mean_af = hl.agg.filter(~hl.is_nan(rows_ht.variant_qc.AF[0]), hl.agg.mean(1 - rows_ht.variant_qc.AF[0])),
                   mean_vqslod = hl.agg.filter(~hl.is_nan(rows_ht.info.VQSLOD), hl.agg.mean(rows_ht.info.VQSLOD)),
                   n_variants = hl.agg.count()))

We convert the results to a Pandas dataframe.

In [14]:
vqsr_stats_pd = result.to_pandas()

[Stage 6:=====================================>                  (25 + 12) / 37]2021-10-27 12:05:52 Hail: INFO: Ordering unsorted dataset with network shuffle


Below we first print the statistics for the SNPs and then for the indels. The rows are sorted by mean genotype quality. The empty filter row (`filter=''`) corresponds to all unfiltered variants.

In [15]:
vqsr_stats_pd[vqsr_stats_pd.filters.isin(['VQSRTrancheSNP50.00to75.00', 'VQSRTrancheSNP75.00to90.00', 'VQSRTrancheSNP90.00to99.00', 'VQSRTrancheSNP99.00to99.90', 'VQSRTrancheSNP99.90to100.00', '', None])].sort_values('mean_vqslod')

,filters,mean_gq,mean_dp,mean_af,mean_vqslod,n_variants
10,VQSRTrancheSNP99.90to100.00,50.122640,31.077643,0.038038,-47.243312,68405
9,VQSRTrancheSNP99.00to99.90,53.528405,23.751122,0.062698,-2.131846,139682
8,VQSRTrancheSNP90.00to99.00,67.947504,26.119766,0.122243,1.311792,111492
7,VQSRTrancheSNP75.00to90.00,68.978084,27.397557,0.126402,2.819369,93294
6,VQSRTrancheSNP50.00to75.00,72.908350,30.275901,0.185178,4.374752,102544
0,,75.856793,35.295114,0.264691,5.999987,159849


In [16]:
vqsr_stats_pd[vqsr_stats_pd.filters.isin(['VQSRTrancheINDEL50.00to75.00', 'VQSRTrancheINDEL75.00to90.00', 'VQSRTrancheINDEL90.00to99.00', 'VQSRTrancheINDEL99.00to99.90', 'VQSRTrancheINDEL99.90to100.00', '', None])].sort_values('mean_vqslod')

,filters,mean_gq,mean_dp,mean_af,mean_vqslod,n_variants
5,VQSRTrancheINDEL99.90to100.00,71.656205,33.762890,0.022044,-8.835908,53479
4,VQSRTrancheINDEL99.00to99.90,53.675363,21.572777,0.030319,-1.810456,46133
3,VQSRTrancheINDEL90.00to99.00,51.933470,19.591386,0.053830,1.435291,56409
2,VQSRTrancheINDEL75.00to90.00,59.114073,24.343697,0.078168,2.729718,56291
1,VQSRTrancheINDEL50.00to75.00,61.357614,28.550483,0.130914,4.307886,24351
0,,75.856793,35.295114,0.264691,5.999987,159849


## Genotype QC

In [17]:
p = hl.plot.histogram(mt.GQ, range=(0, 100))
p.xaxis.axis_label = 'Genotype quality'
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 11:======================================================> (36 + 1) / 37]

### Allelic balance

Note that since we've split the table, all rows are diallelic.

We compute the allelic balance as $AB = \frac{AD[1]}{DP}$. Note that DP is equivalent to $AD[0] + AD[1]$ for a diallelic site.

**NOTE:** It is important that we use the split dataset here.

In [18]:
split_mt = split_mt.annotate_entries(AB = split_mt.AD[1] / split_mt.DP)

In [19]:
hets_mt = split_mt.filter_entries(split_mt.GT.is_het())
p = hl.plot.histogram(hets_mt.AB, range=(0, 1))
p.xaxis.axis_label = 'Allelic balance'
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 12:=======================================================>(73 + 1) / 74]

## Apply filters

Remove all variants failing any of the VQSR filters.

**NOTE:** We must not used the split dataset for filtering VQSR. This is because VQSR consideres all alleles in the site jointly (unless [allele specific annotations](https://gatk.broadinstitute.org/hc/en-us/articles/360035890551?id=9622) have been used).

**FIXME:** be smart about when I use `hl.variant_qc()`.

In [20]:
mt = mt.filter_rows(mt.filters == '')

We also use some hard filters. These are the same as the ones used in the single sample version of the [LinkSeq pipeline](https://github.com/olavurmortensen/linkseq/blob/master/main.nf#L593).

**NOTE:** We here assume that there only two types of variants in the data, SNPs and indels.

In [21]:
mt = mt.filter_rows(hl.if_else(
    hl.is_snp(mt.alleles[0],mt.alleles[1]),
    (mt.info.QD > 2) & (mt.info.SOR < 3) & (mt.info.FS < 60) & (mt.info.MQ > 40) & (mt.info.MQRankSum > -12.5) & (mt.info.ReadPosRankSum > -8),
    (mt.info.QD > 2) & (mt.info.FS < 200) & (mt.info.ReadPosRankSum > -20)))

Filter genotypes with low quality (GQ), using GQ > 20 for SNPs and GQ > 40 for indels.

After the filter some sites may have become invariant. These are removed.

NOTE: Many (or most) of the lower quality genotypes, i.e. the low-end tail of the histogram above, are indels.

In [22]:
mt = mt.filter_entries(hl.if_else(
    hl.is_snp(mt.alleles[0],mt.alleles[1]),
    mt.GQ > 20,
    mt.GQ > 40))

Before we can filter by allelic balance and do HWE filtering, we need to split multi-allelic sites.

In [23]:
# NOTE: permit_shuffle is needed when there are duplicate loci, one of which has multiple alternate alleles. Apparently, this dataset has such sites.
mt = hl.split_multi_hts(mt, permit_shuffle=True)

Filter all heterozygotes with allelic balance outside the range of $]0.25;0.75[$.

In [24]:
mt = mt.annotate_entries(AB = mt.AD[1] / mt.DP)

In [25]:
mt = mt.filter_entries(hl.if_else(
    mt.GT.is_het(),
    (mt.AB > 0.25) & (mt.AB < 0.75),
    True))

Remove variants that significantly deviate from HWE. We use a p-value of $10^{-9}$ for SNPs and $10^{-6}$ for indels. Since our criteria for indels is 1000 times more stringent than for SNPs, we cannot expect the indels to carry information about population structure and inbreeding.

In [26]:
# Update variant statistics.
mt = hl.variant_qc(mt)

mt = mt.filter_rows(hl.if_else(
    hl.is_snp(mt.alleles[0], mt.alleles[1]),
    mt.variant_qc.p_value_hwe > 1e-9,
    mt.variant_qc.p_value_hwe > 1e-6))

Remove singletons.

In [27]:
mt = mt.filter_rows(mt.variant_qc.AC[1] > 1)

In [28]:
variant_counts(mt)

[Stage 13:=======================================================>(73 + 1) / 74]

,
n_variants,148305.000000
snps_fraction,0.841172
insertions_fraction,0.072648
deletions_fraction,0.086181
mnp_fraction,0.000000
complex_fraction,0.000000
star_fraction,0.000000


## Sample QC

Below we see histograms of sample mean genotype quality and genotype depth. Most samples seem to have good depth and quality, although the deviation between samples is quite large. There are some samples with low depth and quality, but we will not worry about these.

In [29]:
mt = hl.sample_qc(mt)

In [30]:
p = hl.plot.histogram(mt.sample_qc.gq_stats.mean, range=(0,100), legend='Mean Sample GQ')
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 14:=======================================================>(73 + 1) / 74]

In [31]:
p = hl.plot.histogram(mt.sample_qc.dp_stats.mean, range=(0,60), legend='Mean Sample DP')
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 17:=======================================================>(73 + 1) / 74]

Below is a histogram of the heterozygote/homozygote ratio.

In [32]:
p = hl.plot.histogram(mt.sample_qc.r_het_hom_var, legend='Het/hom rate')
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 23:=======================================================>(73 + 1) / 74]

Looks like there are a few samples with a lot higher het/hom rate than the rest of the samples. Let's check whether this is due to poor coverage.

In [33]:
het_hom_thres = 3
mt = mt.annotate_cols(high_hom_het=mt.sample_qc.r_het_hom_var > het_hom_thres)

In [34]:
p = hl.plot.scatter(mt.sample_qc.dp_stats.mean, mt.sample_qc.r_het_hom_var, xlabel='DP mean', ylabel='het/hom rate',
                    hover_fields={'Sample': mt.s}, label=mt.high_hom_het)
p.plot_width = 600
p.plot_height = 600
show(p)

[Stage 26:======================================================> (72 + 2) / 74]

The plot above indicates these samples have similar coverage as the other samples, so that doesn't explain the high het/hom rate.

We can also check the MultiQC reports of these samples (file paths below), and we see that they are all of reasonable quality.

In [35]:
high_hethom_samples = mt.filter_cols(mt.sample_qc.r_het_hom_var > het_hom_thres).s.collect()

for sample in high_hethom_samples:
    print('/data/projects/fargen_phase_1/data/single_sample_data/{sample}/multiqc/multiqc_report.html'.format(sample=sample))

[Stage 29:=======================================================>(73 + 1) / 74]

/data/projects/fargen_phase_1/data/single_sample_data/FN000909/multiqc/multiqc_report.html
/data/projects/fargen_phase_1/data/single_sample_data/FN000940/multiqc/multiqc_report.html
/data/projects/fargen_phase_1/data/single_sample_data/FN001018/multiqc/multiqc_report.html
/data/projects/fargen_phase_1/data/single_sample_data/FN001019/multiqc/multiqc_report.html


In the genealogy summary file (`/fargen/fargen_phase_1_utils/multi_sample/joint_genotyping/metadata/genealogy/individuals_summary.csv`), it seems that all these samples have reasonably deep roots in the Faroes. This means we have no reason to suspect this difference is due to these samples being from different populations.

These four samples most likely have high het/hom rate due to poor data quality. One potential reason for this is contamination of the sample in the lab.

We will **discard high het/hom rate samples**, as they may skew further analyses down the line.

In [36]:
mt = mt.filter_cols(mt.sample_qc.r_het_hom_var < het_hom_thres)

Remove sites that may have become invariant after removing these samples.


In [37]:
mt = mt.filter_rows(mt.variant_qc.AC[1] > 1)

In [38]:
n_variants, n_samples = mt.count()
print('Number of variants: ' + str(n_variants))
print('Number of samples: ' + str(n_samples))

[Stage 33:=======================================================>(73 + 1) / 74]

Number of variants: 148305
Number of samples: 469


## Variant counts

In [39]:
variant_counts(mt)

[Stage 34:=======================================================>(73 + 1) / 74]

,
n_variants,148305.000000
snps_fraction,0.841172
insertions_fraction,0.072648
deletions_fraction,0.086181
mnp_fraction,0.000000
complex_fraction,0.000000
star_fraction,0.000000


## Write data to file

In [41]:
if True:
    mt.write(BASE_DIR + '/data/mt/high_quality_variants.mt', overwrite=True)

[Stage 37:======================================================> (36 + 1) / 37]2021-10-27 12:26:13 Hail: INFO: Coerced sorted dataset
[Stage 39:====================================================>   (35 + 2) / 37]2021-10-27 12:26:47 Hail: INFO: wrote matrix table with 148305 rows and 469 columns in 37 partitions to /home/olavur/experiments/2020-11-13_fargen1_exome_analysis/data/mt/high_quality_variants.mt
    Total size: 815.71 MiB
    * Rows/entries: 815.66 MiB
    * Columns: 49.90 KiB
    * Globals: 11.00 B
    * Smallest partition: 1506 rows (6.77 MiB)
    * Largest partition:  4413 rows (25.47 MiB)


## Summary

In this notebook we have:

* Split multi-allelic sites
* Filtered variants failing VQSR filter
* Filtered variants with genotype quality (GQ)
    * GQ > 20 for SNPs
    * GQ > 40 for indels
* Filtered variants with allelic balance outside the range $[0.25, 0.75]$
* Filtered variants failing HWE filter
    * $p > 10^{-9}$ for SNPs
    * $p > 10^{-6}$ for indels
* Removed singletons
* Removed four samples with abnormally high het/hom rate (>3)
* Filtered variants with allele count equal to zero